In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from glob import glob

import yaml

import numpy as np
import xarray as xr
import pandas as pd

import config
import util

In [3]:
cluster, client = util.get_ClusterClient()
cluster.scale(12)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/asm/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/asm/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.205.13:45828,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/asm/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
tracer_flux_mapping = yaml.safe_load(f"""
apo_jena:
  constituent: O2
  flux_file: SFAPO_OCN.carboscope.apo99X_v2021.1x1.19860101-20201231.nc
  flux_variable: SFAPO_OCN
co2_cesm:
  constituent: CO2
  flux_file: SFCO2_OCN.cesm_fosi_smyle.1x1.19860101-20201231.nc
  flux_variable: SFCO2_OCN
co2_gridfed:
  constituent: CO2
  flux_file: SFCO2_FF.GCP-GridFED.v2021.3.1x1.19860101-20201231.nc
  flux_variable: SFCO2_FF
co2_oco2mip:
  constituent: CO2
  flux_file: SFCO2_FF.OCO2-MIP.v2020.1.1x1.19860101-20201231.nc
  flux_variable: SFCO2_FF
co2_somffn:
  constituent: CO2
  flux_file: SFCO2_OCN.MPI-SOM-FFN.1x1.19860101-20201231.nc
  flux_variable: SFCO2_OCN
n2_cesm:
  constituent: N2
  flux_file: SFN2_OCN.cesm_fosi_smyle.1x1.19860101-20201231.nc
  flux_variable: SFN2_OCN
n2_era:
  constituent: N2
  flux_file: SFN2_OCN.era5_shf.1x1.19860101-20201231.nc
  flux_variable: SFN2_OCN
o2_cesm:
  constituent: O2
  flux_file: SFO2_OCN.cesm_fosi_smyle.1x1.19860101-20201231.nc
  flux_variable: SFO2_OCN
o2_gk01r16:
  constituent: O2
  flux_file: SFO2_OCN.gk2001_R2016.1x1.repeat_monclim.19860101-20201231.nc
  flux_variable: SFO2_OCN
o2_gridfed:
  constituent: O2
  flux_file: SFO2_FF.GCP-GridFED.v2021.3.1x1.19860101-20201231.nc
  flux_variable: SFO2_FF
""")

tracer_flux_mapping

{'apo_jena': {'constituent': 'O2',
  'flux_file': 'SFAPO_OCN.carboscope.apo99X_v2021.1x1.19860101-20201231.nc',
  'flux_variable': 'SFAPO_OCN'},
 'co2_cesm': {'constituent': 'CO2',
  'flux_file': 'SFCO2_OCN.cesm_fosi_smyle.1x1.19860101-20201231.nc',
  'flux_variable': 'SFCO2_OCN'},
 'co2_gridfed': {'constituent': 'CO2',
  'flux_file': 'SFCO2_FF.GCP-GridFED.v2021.3.1x1.19860101-20201231.nc',
  'flux_variable': 'SFCO2_FF'},
 'co2_oco2mip': {'constituent': 'CO2',
  'flux_file': 'SFCO2_FF.OCO2-MIP.v2020.1.1x1.19860101-20201231.nc',
  'flux_variable': 'SFCO2_FF'},
 'co2_somffn': {'constituent': 'CO2',
  'flux_file': 'SFCO2_OCN.MPI-SOM-FFN.1x1.19860101-20201231.nc',
  'flux_variable': 'SFCO2_OCN'},
 'n2_cesm': {'constituent': 'N2',
  'flux_file': 'SFN2_OCN.cesm_fosi_smyle.1x1.19860101-20201231.nc',
  'flux_variable': 'SFN2_OCN'},
 'n2_era': {'constituent': 'N2',
  'flux_file': 'SFN2_OCN.era5_shf.1x1.19860101-20201231.nc',
  'flux_variable': 'SFN2_OCN'},
 'o2_cesm': {'constituent': 'O2',
  'f

In [5]:
USER = os.environ['USER']
dirout = './data'

flux_file_dir = f'/glade/work/{USER}/sno-analysis/sno-flux-products'

convert = dict(
    CO2={'factor': 1e-15 * 86400.0 * 365.25, 'units': 'Pmol/yr'},
    O2={'factor': 1e-15 * 86400.0 * 365.25, 'units': 'Pmol/yr'},
    APO={'factor': 1e-15 * 86400.0 * 365.25, 'units': 'Pmol/yr'},    
    N2={'factor': 1e-15 * 86400.0 * 365.25, 'units': 'Pmol/yr'},
)

index = []
columns = []
data = []
for tracer, info in tracer_flux_mapping.items():
    
    flux_file = f"{flux_file_dir}/{info['flux_file']}"
    assert os.path.exists(flux_file), f'missing {flux_file}'
    constituent = info['constituent']
    
    v = info['flux_variable']
          
    scale_by = convert[constituent]['factor']
    units = convert[constituent]['units']
    columns.append(f'{v}.{tracer} [{units}]')
    
    with xr.open_dataset(flux_file, chunks={'time': 100}) as ds:
        total = (ds[v] * ds.area).sum(['lat', 'lon'])
        total *= scale_by
        data.append(total.values)
        if not len(index):
            index = total.time.values
        else:
            assert (index == total.time.values).all()

df = pd.DataFrame(data=np.array(data).T, index=index, columns=columns)
df            

,SFAPO_OCN.apo_jena [Pmol/yr],SFCO2_OCN.co2_cesm [Pmol/yr],SFCO2_FF.co2_gridfed [Pmol/yr],SFCO2_FF.co2_oco2mip [Pmol/yr],SFCO2_OCN.co2_somffn [Pmol/yr],SFN2_OCN.n2_cesm [Pmol/yr],SFN2_OCN.n2_era [Pmol/yr],SFO2_OCN.o2_cesm [Pmol/yr],SFO2_OCN.o2_gk01r16 [Pmol/yr],SFO2_FF.o2_gridfed [Pmol/yr]
1986-01-01 12:00:00,0.000000,-0.218934,0.528720,0.000000,-0.130405,0.033259,0.476587,0.144992,0.653724,-0.733670
1986-01-02 12:00:00,0.000000,-0.213774,0.529393,0.000000,-0.130006,0.029607,0.473220,0.231176,0.653285,-0.734536
1986-01-03 12:00:00,0.000000,-0.205181,0.530066,0.000000,-0.129607,0.025954,0.469853,0.382042,0.652845,-0.735402
1986-01-04 12:00:00,0.000000,-0.245267,0.530738,0.000000,-0.129208,0.022302,0.466486,0.056199,0.652405,-0.736268
1986-01-05 12:00:00,0.000000,-0.242441,0.531411,0.000000,-0.128809,0.018650,0.463120,-0.011542,0.651965,-0.737134
...,...,...,...,...,...,...,...,...,...,...
2020-12-27 12:00:00,1.423361,-0.381446,0.898174,0.842327,-0.265362,0.076151,0.408909,0.233372,0.655923,-1.241463
2020-12-28 12:00:00,1.457859,-0.343846,0.899931,0.899796,-0.266242,0.076168,0.410834,0.391277,0.655483,-1.244030
2020-12-29 12:00:00,1.489718,-0.353151,0.901688,0.903971,-0.267122,0.076185,0.412759,0.406275,0.655044,-1.246597
2020-12-30 12:00:00,1.518499,-0.436713,0.903445,0.903898,-0.268002,0.076202,0.414685,0.237832,0.654604,-1.249164


In [6]:
df.to_csv(f'{dirout}/apo-forward-flux-products-daily-totals.csv', index_label='time')